In [1]:
import tensorflow as tf

## 0.前言

常见的误差函数：
* 均方差
* 交叉熵
* KL散度
* Hinge Loss

## 1.均方差
Mean Squared Error简称MSE，将输出向量和真是向量映射到笛卡尔坐标系的两个点上，通过计算这两个点之间的欧式距离来衡量两个向量之间的差距：
${\rm MSE}(y,o)\triangleq\frac{1}{d_{out}}\Sigma^{d_{\rm out}}_{i=1}(y_i-o_i)^2$
MSE误差值总是大于等于0，当达到最小值0时，输出等于真实标签，此时网络参数达到最优状态。
其广泛应用在回归问题中，实际上分类问题也可使用。

TensorFlow中实现：

In [2]:
o=tf.random.normal([2,10])
y=tf.constant([1,3])
y_onehot=tf.one_hot(y,depth=10)
loss=tf.keras.losses.MSE(y_onehot,o) # 返回每个样本的MSE
print(loss)
loss=tf.reduce_mean(loss) # 计算batch均方差
print(loss)

tf.Tensor([1.2540859  0.62131596], shape=(2,), dtype=float32)
tf.Tensor(0.9377009, shape=(), dtype=float32)


也可以使用层方式完成，对应的类为`tf.keras.losses.MeanSquaredError()`，调用`__call__`函数即可完成前向计算，代码如下：

In [3]:
criteon=tf.keras.losses.MeanSquaredError()
loss=criteon(y_onehot,o) # 其直接计算batch均方差
print(loss)

tf.Tensor(0.9377009, shape=(), dtype=float32)


## 2.交叉熵

### 2.1.信息熵简介

1948年，Claude Shannon将热力学中熵的概念引入到信息论中，用来衡量信息的不确定度。熵在信息学科中叫信息熵，熵越大，代表不确定性越大，信息量也就越大。
某个分布$P(i)$的熵定义为：<br>
$H(P)\triangleq-\Sigma_iP(i){\rm log}_2P(i)$
<br>
实际上，$H(P)$也可以使用其他底数的log函数计算。

举例：
对于4分类问题，如果某个样本的真实标签是第4类，那么标签的One-hot编码为[0,0,0,1]，即这张图片的分类是唯一确定的，它属于第4类的概率$P(y为4|x)=1$，不确定性为0，其熵可以计算：<br>
$-0\cdot {\rm log}_20-0\cdot {\rm log}_20-0\cdot {\rm log}_20-1\cdot {\rm log}_21=0$

如果它预测的概率分布是[.1,.1,.1,.7]，其熵计算：<br>
$-0.1\cdot {\rm log}_20.1-0.1\cdot {\rm log}_20.1-0.1\cdot{\rm log}_20.1-0.7\cdot{\rm log}_20.7\approx1.356$
<br>可见其不确定性比前面确定性事件的不确定性大。

由于$P(i)\in[0,1],{\rm log}_2P(i)\le0$，因此熵$H(P)$总是大于等于0。当熵取得最小值0时，不确定性为0。分类问题的One-hot编码的分布就是熵为0的典型例子。<br>
在TensorFlow中，可利用`tf.math.log`来计算熵。

### 2.2.交叉熵简介
基于熵引出交叉熵(Cross Entropy)的定义：<br>
$H(p\parallel q)\triangleq-\Sigma_ip(i){\rm log}_2q(i)$<br>
通过变换，交叉熵可以分解为$p$的熵$H(p)$和$p$与$q$的KL散度(Kullback-Leibler Divergence)的和：<br>
$H(p\parallel q)=H(p)+D_{KL}(p\parallel q)$<br>
其中KL定义为：<br>
$D_{KL}(p\parallel q)=\Sigma_ip(i){\rm log}\lgroup\frac{p(i)}{q(i)}\rgroup$<br>
KL散度是Solomon Kullback和Richard A. Leibler在1951年提出的用于衡量2个分布之间距离的指标。$p=q$时，$D_{KL}(p\parallel q)$取得最小值0；$p$与$q$之间的差距越大，$D_{KL}(p\parallel q)$也越大。<br>
需要注意的是，交叉熵和KL散度都不是对称的，即：<br>
$H(p\parallel q)\neq H(q\parallel p)$<br>
$D_{KL}(p\parallel q)\neq D_{KL}(q\parallel p)$<br>

交叉熵可以很好地衡量2个分布之间的"距离"。特别地，当分类问题中$y$的编码分布$p$采用One-hot编码$y$时， $H(p)=0$，此时：<br>
$H(p\parallel q)=H(p)+D_{KL}(p\parallel q)=D_{KL}(p\parallel q)$<br>
退化到真实标签分布$y$与输出概率分布$o$之间的KL散度上。<br>

根据KL散度的定义，我们推导分类问题中交叉熵的计算表达式：<br>
$
\begin{align}
H(p\parallel q)&=D_{KL}(p\parallel q)=\Sigma_jy_j{\rm log}(\frac{y_j}{o_j})\\
&=1\cdot{\rm log}\frac1{o_i}+\Sigma_{j\ne i}0\cdot{\rm log}(\frac0{o_j}) \\
&=-{\rm log}o_i
\end{align}
$

其中$i$为One-hot编码中为1的索引号，也是当前输入的真实类别。可以看到，$\cal L$只与真实类别$i$上的概率$o_i$有关。对应概率$o_i$越大，$H(p\parallel q)$越小。
当对应类别上的概率为1时，交叉熵$H(p\parallel q)$取得最小值0。<br>
因此最小化交叉熵损失函数的过程就是最大化正确类别的预测概率的过程。

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid

